In [101]:
import pickle
from pathlib import Path
import numpy as np
import statistics
import numpy as np
import sympy
import matplotlib.pyplot as plt
import base64
import pickle
from sympy import symbols, MatrixSymbol, lambdify, Matrix, pprint

from scipy.optimize import minimize
from matplotlib.ticker import FuncFormatter
from sympy import symbols, MatrixSymbol, lambdify
from matplotlib import cm
import random
import matplotlib.colors as mcolors
import scipy
import time
from pathlib import Path
import os
import ast
import pandas as pd
from pathlib import Path
from matplotlib.ticker import ScalarFormatter


import pennylane as qml
from functools import partial
from qiskit.circuit.library import *
from qiskit import *
from qiskit.quantum_info import *
import pickle
from pathlib import Path
import numpy as np
import statistics
from pennylane.wires import Wires
import matplotlib.cm as cm
import base64
from qiskit import *
from qiskit.quantum_info import *
import math

In [102]:
from ANALYSIS_SPECIFIC_CONFIG import is_valid_pickle_file,spread_per_sample_vectorized,spread_pooling_vectorized, load_and_clean_pickle, extract_Nr, extract_trotter_step

def compute_single_draw_stats(
    eigvals,
    full_qfim_mat,
    threshold=1e-10,
    spread_methods=("variance", "mad"),
    ddof=1,
    scale="normal",
    gamma=1.0,
    n=1,
    V_theta=1.0,
    n_ctrl=None,
    n_reserv=None,
    trotter_step=None,
):
    """
    Compute QFIM (or DQFIM) statistics for a SINGLE set of eigenvalues (one draw),
    and also compute an effective dimension from the provided full QFIM matrix.
    
    Returned dictionary includes:
      - "draw_rank"
      - "var_all_eigenvalues"
      - "var_nonzero_eigenvalues"
      - "trace_eigenvalues"
      - "var_all_normalized_by_param_count"
      - "trace_normalized_by_rank"
      - "var_nonzero_log"
      - "trace_normalized_by_param_count"
      - "ipr_deff_raw"        (raw IPR measure)
      - "ipr_deff_norm"       (IPR computed on trace-normalized eigenvalues)
      - "abbas_deff_raw"      (sum(log(1 + alpha*λ)) on raw eigenvalues)
      - "abbas_deff_norm"     (sum(log(1 + alpha*λ)) on trace-normalized eigenvalues)
      - "effective_dimension" (computed from the trace-normalized full QFIM)
      - "spread_metric_{method}" for each method in spread_methods.
    
    Parameters
    ----------
    eigvals : array-like
        Eigenvalues for this single QFIM (or DQFIM) draw.
    full_qfim_mat : array-like (2D)
        The full QFIM matrix for this draw.
    threshold : float
        Zero out eigenvalues below this threshold.
    spread_methods : tuple of str
        Methods for "spread-of-log" metrics.
    ddof : int
        Degrees of freedom for variance computations.
    scale : str
        Scale indicator for spread metrics.
    gamma : float
        Scaling parameter in the Abbas formula (typically in (0,1]).
    n : int
        Number of data samples used in the Abbas formula.
    V_theta : float
        Volume factor (typically 1.0).
    n_ctrl, n_reserv, trotter_step : optional
        Additional metadata.
    
    Returns
    -------
    stats_dict : dict
        Dictionary of computed statistics.
    """
    # Ensure eigvals is a 1D NumPy array.
    arr = np.array(eigvals, dtype=float)
    if arr.ndim != 1:
        arr = arr.flatten()
    # Zero out small eigenvalues.
    arr = np.where(arr < threshold, 0.0, arr)
    
    # --- 1) Basic stats ---
    draw_rank = np.count_nonzero(arr)
    var_all_eigenvalues = np.var(arr, ddof=ddof)
    # Variance on nonzero values using Boolean indexing.
    nonzero = arr[arr > threshold]
    var_nonzero_eigenvalues = np.var(nonzero, ddof=ddof) if nonzero.size > 1 else 0.0
    var_nonzero_log = np.log(var_nonzero_eigenvalues) if var_nonzero_eigenvalues > 0 else -np.inf
    trace_eigenvalues = np.sum(arr)
    min_nonzero_eigenvalue = np.min(nonzero) 
    
    var_normalized_by_param_count = var_all_eigenvalues / len(arr)
    var_nonzero_normalized_by_rank = var_nonzero_eigenvalues / draw_rank
    var_normalized_by_rank = var_all_eigenvalues / draw_rank
    trace_normalized_by_rank = (trace_eigenvalues / draw_rank) if draw_rank > 0 else 0.0
    trace_normalized_by_param_count = trace_eigenvalues / len(arr)
    
    # --- 2) IPR-based dimensions ---
    # Raw IPR: (trace^2) / (sum of squares)
    sum_of_squares = np.sum(arr**2)
    ipr_deff_raw = (trace_eigenvalues**2) / sum_of_squares if sum_of_squares > 0 else 0.0
    
    # Normalized IPR: using normalized eigenvalues (p = λ/trace)
    if trace_eigenvalues > 0:
        arr_norm = arr / trace_eigenvalues
        sum_norm_sq = np.sum(arr_norm**2)
        ipr_deff_norm = 1.0 / sum_norm_sq if sum_norm_sq > 0 else 0.0
    else:
        arr_norm = None
        ipr_deff_norm = 0.0
    
    # --- 3) Abbas-based dimensions ---
    # Compute alpha = (gamma * n) / (2*log(n)) if n>1, else use limit.
    if n > 1 and math.log(n) != 0.0:
        alpha = (gamma * n) / (2.0 * math.log(n))
    else:
        alpha = 0.0
    # Raw Abbas: computed on original eigenvalues.
    abbas_deff_raw = np.sum(np.log(np.maximum(1.0 + alpha * arr, 1e-15)))
    # Normalized Abbas: computed on trace-normalized eigenvalues.
    if arr_norm is not None:
        abbas_deff_norm = np.sum(np.log(np.maximum(1.0 + alpha * arr_norm, 1e-15)))
    else:
        abbas_deff_norm = 0.0
    
    # --- 4) Effective dimension from the full QFIM ---
    # Normalize the full QFIM by its trace BEFORE diagonalizing.
    F = np.array(full_qfim_mat, dtype=complex)
    trF = np.trace(F)
    if trF > 0:
        F_hat = F / trF
        eigs_F = np.linalg.eigvalsh(F_hat)  # Eigenvalues of the normalized full QFIM.
        eps = 1e-12
        # Here, effective dimension is computed from the normalized spectrum p_i.
        # If n > 1, use the standard formula; if n == 1, use the limit:
        if n > 1 and math.log(n) != 0.0:
            z = 0.5 * np.sum(np.log(1.0 + n * eigs_F + eps))
            effective_dimension = (2.0 / np.log(n)) * z
        else:
            # For n == 1, define effective dimension as the sum_i p_i/(1+p_i)
            effective_dimension = np.sum(eigs_F / (1.0 + eigs_F))
    else:
        effective_dimension = 0.0
    
    # --- 5) Spread-of-log metrics ---
    # Reshape arr into a 1-row 2D array for external functions.
    arr_2d = arr.reshape(1, -1)
    spread_metrics = {}
    for method in spread_methods:
        per_draw = spread_per_sample_vectorized(arr_2d, method=method, threshold=threshold, ddof=ddof, scale=scale)
        spread_metrics[f"spread_metric_{method}"] = per_draw[0] if per_draw.size > 0 else 0.0
    
    # --- 6) Build final dictionary ---
    stats_dict = {
        # Basic stats
        "draw_rank": draw_rank,
        "var_all_eigenvalues": var_all_eigenvalues,
        "var_nonzero_eigenvalues": var_nonzero_eigenvalues,
        "trace_eigenvalues": trace_eigenvalues,
        "var_all_normalized_by_param_count": var_normalized_by_param_count,
        "var_all_normalized_by_rank": var_normalized_by_rank,
        "var_nonzero_normalized_by_rank":var_nonzero_normalized_by_rank,
        "trace_normalized_by_rank": trace_normalized_by_rank,
        "trace_normalized_by_param_count": trace_normalized_by_param_count,
        "var_nonzero_log": var_nonzero_log,
        
        # IPR-based dimensions
        "ipr_deff_raw": ipr_deff_raw,
        "ipr_deff_norm": ipr_deff_norm,
        
        # Abbas-based dimensions
        "abbas_deff_raw": abbas_deff_raw,
        "abbas_deff_norm": abbas_deff_norm,
        
        # Effective dimension computed from the full QFIM (trace-normalized)
        "d_eff": effective_dimension,
        # Minimum nonzero eigenvalue (above threshold)
        "min_nonzero_eigenvalue": min_nonzero_eigenvalue,
    }
    stats_dict.update(spread_metrics)
    
    return stats_dict


## <b>QFIM</b>, T = 2

In [92]:
T = 2
sample_range_label ='normal_.25pi'
Nc = 1
Nr = 1
# file_path = f'/Users/sophieblock/QRCCapstone/parameter_analysis_directory/QFIM_results/analog/Nc_{Nc}/sample_{sample_range_label}/1xK/Nr_{Nr}/trotter_step_{T}/data.pickle'
file_path = f'/Users/sophieblock/QRCCapstone/parameter_analysis_directory/QFIM_global_results/analog_model_DQFIM/Nc_{Nc}/sample_{sample_range_label}/1xK/Nr_{Nr}/trotter_step_{T}/L_10/data.pickle'

with open(file_path, 'rb') as f:
    all_tests_data = pickle.load(f)
fixed_params_dict = 'fixed_params0'
test_data = all_tests_data[fixed_params_dict]

In [93]:
threshold=1e-8
# Existing dictionaries
params_sorted_by_qfim_trace = {}
params_sorted_by_qfim_eigvals = {}
params_sorted_by_qfim_var, params_sorted_by_qfim_nonzero_var = {}, {}
params_sorted_by_qfim_mineig = {}
params_sorted_by_qfim_entropy = {}
key_and_params = {}

# NEW dictionaries for IPR/Abbas or single-draw stats
params_sorted_by_ipr_raw = {}
params_sorted_by_ipr_norm = {}
params_sorted_by_abbas_raw = {}
params_sorted_by_abbas_norm = {}
params_sorted_rank = {}
params_sorted_spread_var, params_sorted_spread_mad = {}, {}

# NEW dictionaries for the additional normalized metrics:
params_sorted_by_var_all_normalized_by_param_count,params_sorted_by_var_all_normalized_by_rank = {},{}
params_sorted_by_var_nonzero_normalized_by_rank = {}
params_sorted_by_var_nonzero_log = {}
params_sorted_by_trace_normalized_by_param_count = {}

params_sorted_rank, params_sorted_deff= {},{}

for trainable_params_dict in test_data.keys():
    results = test_data[trainable_params_dict]
    qfi_eigvals = results['qfim_eigvals']
    stats = compute_single_draw_stats(qfi_eigvals,results['qfim'],  threshold=threshold, gamma=1.0, n=1)
    
    # Extract metrics from stats
    rank = stats["draw_rank"]
    deff = stats['d_eff']
    # print(deff)
    trace = stats["trace_eigenvalues"]
    variance = stats["var_all_eigenvalues"]
    variance_nonzero = stats["var_nonzero_eigenvalues"]
    ipr_raw = stats["ipr_deff_raw"]
    ipr_norm = stats["ipr_deff_norm"]
    abbas_raw = stats["abbas_deff_raw"]
    abbas_norm = stats["abbas_deff_norm"]
    spread_mad = stats["spread_metric_mad"]
    spread_var = stats["spread_metric_variance"]
    
    #  normalized metrics 
    var_all_norm = stats["var_all_normalized_by_param_count"]
    var_all_norm_rank = stats["var_all_normalized_by_rank"]
    var_nonzero_norm = stats["var_nonzero_normalized_by_rank"]
    var_nonzero_log = stats["var_nonzero_log"]
    trace_norm_param = stats["trace_normalized_by_param_count"]
    
    # Store existing metrics
    params_sorted_deff[trainable_params_dict] = deff
    params_sorted_by_qfim_trace[trainable_params_dict] = float(trace)
    params_sorted_by_qfim_var[trainable_params_dict] = float(variance)
    params_sorted_by_qfim_nonzero_var[trainable_params_dict] = float(variance_nonzero)
   
    params_sorted_by_qfim_mineig[trainable_params_dict] = (rank, float(stats["min_nonzero_eigenvalue"]))

    params_sorted_spread_mad[trainable_params_dict] = spread_mad
    params_sorted_spread_var[trainable_params_dict] = spread_var
    
    params_sorted_by_ipr_raw[trainable_params_dict] = ipr_raw
    params_sorted_by_ipr_norm[trainable_params_dict] = ipr_norm
    params_sorted_by_abbas_raw[trainable_params_dict] = abbas_raw
    params_sorted_by_abbas_norm[trainable_params_dict] = abbas_norm
    params_sorted_rank[trainable_params_dict] = rank
    
    try:
        ent = float(np.mean(results['entropies']))
    except KeyError:
        ent = float(results['entropy'])
    params_sorted_by_qfim_entropy[trainable_params_dict] = ent
    
    key_and_params[trainable_params_dict] = results['trainable_params']
    
    # Store the new normalized metrics
    params_sorted_by_var_all_normalized_by_param_count[trainable_params_dict] = float(var_all_norm)
    params_sorted_by_var_all_normalized_by_rank[trainable_params_dict] = float(var_all_norm_rank)
    params_sorted_by_var_nonzero_normalized_by_rank[trainable_params_dict] = float(var_nonzero_norm)
    params_sorted_by_var_nonzero_log[trainable_params_dict] = float(var_nonzero_log)
    params_sorted_by_trace_normalized_by_param_count[trainable_params_dict] = float(trace_norm_param)

# print(params_sorted_by_qfim_entropy)

# print(key_and_params.keys())
number_of_tests = len(test_data)

sorted_qfim_entropy = sorted(params_sorted_by_qfim_entropy.items(), key=lambda x: round(float(x[1]), 6))
sorted_qfim_var = sorted(params_sorted_by_qfim_var.items(), key=lambda x: float(x[1]))
sorted_qfim_var_normalized_by_rank = sorted(params_sorted_by_var_all_normalized_by_rank.items(), key=lambda x: float(x[1]))
sorted_qfim_var_nonzero = sorted(params_sorted_by_qfim_nonzero_var.items(), key=lambda x: float(x[1]))
sorted_qfim_trace = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: float(x[1]))
# params_sorted_by_abbas_raw
sorted_qfim_deff = sorted(params_sorted_deff.items(), key=lambda x: float(x[1]))
sorted_qfim_abbas_d_eff = sorted(params_sorted_by_abbas_raw.items(), key=lambda x: float(x[1]))
sorted_qfim_ipr_d_eff = sorted(params_sorted_by_ipr_raw.items(), key=lambda x: float(x[1]))
# Calculate the number of parameters to select
number_of_tests = len(test_data)
sorted_qfim_spread_var = sorted(params_sorted_spread_var.items(), key=lambda x: float(x[1]))
sorted_qfim_mineig = sorted(
    params_sorted_by_qfim_mineig.items(), 
    key=lambda x: (x[1][0], x[1][1])
)
def round_tuples2(lst, ndigits=2):
    """Convert each (key, value) in lst to (key, formatted_value).
    
    If the rounded value is less than or equal to ndigits, it is formatted in scientific notation (e).
    Otherwise, it is formatted in fixed-point notation (f).
    """
    formatted = []
    for k, v in lst:
        val = round(float(v), ndigits)
        if val <= ndigits:
            formatted_value = f"{v:.{ndigits+1}e}"
        else:
            formatted_value = f"{val:.{ndigits}f}"
        formatted.append((k, formatted_value))
    return formatted
def format_metric(val, ndigits=2):
    """
    Format a numeric value such that if its rounded value is less than or equal to ndigits,
    it is printed in scientific notation; otherwise in fixed-point notation.
    """
    # Convert to float in case it's not already.
    val = float(val)
    rounded_val = round(val, ndigits)
    if rounded_val <= ndigits:
        return f"{val:.{ndigits+1}e}"
    else:
        return f"{val:.{ndigits}f}"
num_print = 3
print(f"sorted_qfim_trace: \n - min: {round_tuples2(sorted_qfim_trace)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_trace[::-1])[:num_print]}\n")

print(f"sorted_qfim_var: \n - min: {round_tuples2(sorted_qfim_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_var_normalized_by_rank[::-1])[:num_print]}\n")

# print(f"sorted_qfim_mineig: \n - min: {round_tuples2(sorted_qfim_mineig)[:num_print]}")
# print(f" - max: {round_tuples2(sorted_qfim_mineig[::-1])[:num_print]}\n")

print(f"sorted_qfim_spread_var: \n - min: {round_tuples2(sorted_qfim_spread_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_spread_var[::-1])[:num_print]}\n")

print(f"sorted_qfim_deff: \n - min: {round_tuples2(sorted_qfim_deff)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_deff[::-1])[:num_print]}\n")
print("sorted_qfim_mineig:\n - min: ")
for key, (rnk, mineig) in sorted_qfim_mineig[:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")
print(f" - max: ")
for key, (rnk, mineig) in sorted_qfim_mineig[::-1][:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")

sorted_qfim_trace: 
 - min: [('test32', '6.775e-05'), ('test120', '1.337e-03'), ('test143', '8.949e-03')]
 - max: [('test9', '6.25'), ('test48', '5.86'), ('test15', '5.43')]

sorted_qfim_var: 
 - min: [('test32', '6.470e-10'), ('test120', '2.460e-07'), ('test143', '1.142e-05')]
 - max: [('test9', '1.833e+00'), ('test48', '1.597e+00'), ('test15', '1.398e+00')]

sorted_qfim_spread_var: 
 - min: [('test193', '1.353e+00'), ('test1', '1.607e+00'), ('test58', '2.28')]
 - max: [('test197', '33.52'), ('test129', '29.84'), ('test123', '26.47')]

sorted_qfim_deff: 
 - min: [('test197', '5.005e-01'), ('test81', '5.005e-01'), ('test112', '5.005e-01')]
 - max: [('test193', '6.483e-01'), ('test1', '6.229e-01'), ('test121', '6.047e-01')]

sorted_qfim_mineig:
 - min: 
  test32: rank = 2, min eigenvalue = 3.8748316910641734e-07
  test143: rank = 3, min eigenvalue = 8.285804824481602e-07
  test120: rank = 3, min eigenvalue = 2.601603910079575e-06
 - max: 
  test100: rank = 3, min eigenvalue = 0.04975012

In [97]:
threshold=1e-8
# Existing dictionaries
params_sorted_by_qfim_trace = {}
params_sorted_by_qfim_eigvals = {}
params_sorted_by_qfim_var, params_sorted_by_qfim_nonzero_var = {}, {}
params_sorted_by_qfim_mineig = {}
params_sorted_by_qfim_entropy = {}
key_and_params = {}

# NEW dictionaries for IPR/Abbas or single-draw stats
params_sorted_by_ipr_raw = {}
params_sorted_by_ipr_norm = {}
params_sorted_by_abbas_raw = {}
params_sorted_by_abbas_norm = {}
params_sorted_rank = {}
params_sorted_spread_var, params_sorted_spread_mad = {}, {}

# NEW dictionaries for the additional normalized metrics:
params_sorted_by_var_all_normalized_by_param_count,params_sorted_by_var_all_normalized_by_rank = {},{}
params_sorted_by_var_nonzero_normalized_by_rank = {}
params_sorted_by_var_nonzero_log = {}
params_sorted_by_trace_normalized_by_param_count = {}

params_sorted_rank, params_sorted_deff= {},{}

for trainable_params_dict in test_data.keys():
    results = test_data[trainable_params_dict]
    qfi_eigvals = results['qfim_eigvals']
    stats = compute_single_draw_stats(qfi_eigvals,results['qfim'],  threshold=threshold, gamma=1.0, n=1)
    
    # Extract metrics from stats
    rank = stats["draw_rank"]
    deff = stats['d_eff']
    # print(deff)
    trace = stats["trace_eigenvalues"]
    variance = stats["var_all_eigenvalues"]
    variance_nonzero = stats["var_nonzero_eigenvalues"]
    ipr_raw = stats["ipr_deff_raw"]
    ipr_norm = stats["ipr_deff_norm"]
    abbas_raw = stats["abbas_deff_raw"]
    abbas_norm = stats["abbas_deff_norm"]
    spread_mad = stats["spread_metric_mad"]
    spread_var = stats["spread_metric_variance"]
    
    #  normalized metrics 
    var_all_norm = stats["var_all_normalized_by_param_count"]
    var_all_norm_rank = stats["var_all_normalized_by_rank"]
    var_nonzero_norm = stats["var_nonzero_normalized_by_rank"]
    var_nonzero_log = stats["var_nonzero_log"]
    trace_norm_param = stats["trace_normalized_by_param_count"]
    
    # Store existing metrics
    params_sorted_deff[trainable_params_dict] = deff
    params_sorted_by_qfim_trace[trainable_params_dict] = float(trace)
    params_sorted_by_qfim_var[trainable_params_dict] = float(variance)
    params_sorted_by_qfim_nonzero_var[trainable_params_dict] = float(variance_nonzero)
   
    params_sorted_by_qfim_mineig[trainable_params_dict] = (rank, float(stats["min_nonzero_eigenvalue"]))

    params_sorted_spread_mad[trainable_params_dict] = spread_mad
    params_sorted_spread_var[trainable_params_dict] = spread_var
    
    params_sorted_by_ipr_raw[trainable_params_dict] = ipr_raw
    params_sorted_by_ipr_norm[trainable_params_dict] = ipr_norm
    params_sorted_by_abbas_raw[trainable_params_dict] = abbas_raw
    params_sorted_by_abbas_norm[trainable_params_dict] = abbas_norm
    params_sorted_rank[trainable_params_dict] = rank
    
    try:
        ent = float(np.mean(results['entropies']))
    except KeyError:
        ent = float(results['entropy'])
    params_sorted_by_qfim_entropy[trainable_params_dict] = ent
    
    key_and_params[trainable_params_dict] = results['trainable_params']
    
    # Store the new normalized metrics
    params_sorted_by_var_all_normalized_by_param_count[trainable_params_dict] = float(var_all_norm)
    params_sorted_by_var_all_normalized_by_rank[trainable_params_dict] = float(var_all_norm_rank)
    params_sorted_by_var_nonzero_normalized_by_rank[trainable_params_dict] = float(var_nonzero_norm)
    params_sorted_by_var_nonzero_log[trainable_params_dict] = float(var_nonzero_log)
    params_sorted_by_trace_normalized_by_param_count[trainable_params_dict] = float(trace_norm_param)

# print(params_sorted_by_qfim_entropy)

# print(key_and_params.keys())
number_of_tests = len(test_data)

sorted_qfim_entropy = sorted(params_sorted_by_qfim_entropy.items(), key=lambda x: round(float(x[1]), 6))
sorted_qfim_var = sorted(params_sorted_by_qfim_var.items(), key=lambda x: float(x[1]))
sorted_qfim_var_normalized_by_rank = sorted(params_sorted_by_var_all_normalized_by_rank.items(), key=lambda x: float(x[1]))
sorted_qfim_var_nonzero = sorted(params_sorted_by_qfim_nonzero_var.items(), key=lambda x: float(x[1]))
sorted_qfim_trace = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: float(x[1]))
# params_sorted_by_abbas_raw
sorted_qfim_deff = sorted(params_sorted_deff.items(), key=lambda x: float(x[1]))
sorted_qfim_abbas_d_eff = sorted(params_sorted_by_abbas_raw.items(), key=lambda x: float(x[1]))
sorted_qfim_ipr_d_eff = sorted(params_sorted_by_ipr_raw.items(), key=lambda x: float(x[1]))
# Calculate the number of parameters to select
number_of_tests = len(test_data)
sorted_qfim_spread_var = sorted(params_sorted_spread_var.items(), key=lambda x: float(x[1]))
sorted_qfim_mineig = sorted(
    params_sorted_by_qfim_mineig.items(), 
    key=lambda x: (x[1][0], x[1][1])
)
def round_tuples2(lst, ndigits=2):
    """Convert each (key, value) in lst to (key, formatted_value).
    
    If the rounded value is less than or equal to ndigits, it is formatted in scientific notation (e).
    Otherwise, it is formatted in fixed-point notation (f).
    """
    formatted = []
    for k, v in lst:
        val = round(float(v), ndigits)
        if val <= ndigits:
            formatted_value = f"{v:.{ndigits+1}e}"
        else:
            formatted_value = f"{val:.{ndigits}f}"
        formatted.append((k, formatted_value))
    return formatted
def format_metric(val, ndigits=2):
    """
    Format a numeric value such that if its rounded value is less than or equal to ndigits,
    it is printed in scientific notation; otherwise in fixed-point notation.
    """
    # Convert to float in case it's not already.
    val = float(val)
    rounded_val = round(val, ndigits)
    if rounded_val <= ndigits:
        return f"{val:.{ndigits+1}e}"
    else:
        return f"{val:.{ndigits}f}"
num_print = 3
print(f"sorted_qfim_trace: \n - min: {round_tuples2(sorted_qfim_trace)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_trace[::-1])[:num_print]}\n")

print(f"sorted_qfim_var: \n - min: {round_tuples2(sorted_qfim_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_var_normalized_by_rank[::-1])[:num_print]}\n")

# print(f"sorted_qfim_mineig: \n - min: {round_tuples2(sorted_qfim_mineig)[:num_print]}")
# print(f" - max: {round_tuples2(sorted_qfim_mineig[::-1])[:num_print]}\n")

print(f"sorted_qfim_spread_var: \n - min: {round_tuples2(sorted_qfim_spread_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_spread_var[::-1])[:num_print]}\n")

print(f"sorted_qfim_deff: \n - min: {round_tuples2(sorted_qfim_deff)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_deff[::-1])[:num_print]}\n")
print("sorted_qfim_mineig:\n - min: ")
for key, (rnk, mineig) in sorted_qfim_mineig[:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")
print(f" - max: ")
for key, (rnk, mineig) in sorted_qfim_mineig[::-1][:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")

sorted_qfim_trace: 
 - min: [('test32', '6.775e-05'), ('test120', '1.337e-03'), ('test143', '8.949e-03')]
 - max: [('test9', '6.25'), ('test48', '5.86'), ('test15', '5.43')]

sorted_qfim_var: 
 - min: [('test32', '6.470e-10'), ('test120', '2.460e-07'), ('test143', '1.142e-05')]
 - max: [('test9', '1.833e+00'), ('test48', '1.597e+00'), ('test15', '1.398e+00')]

sorted_qfim_spread_var: 
 - min: [('test193', '1.353e+00'), ('test1', '1.607e+00'), ('test58', '2.28')]
 - max: [('test197', '33.52'), ('test129', '29.84'), ('test123', '26.47')]

sorted_qfim_deff: 
 - min: [('test197', '5.005e-01'), ('test81', '5.005e-01'), ('test112', '5.005e-01')]
 - max: [('test193', '6.483e-01'), ('test1', '6.229e-01'), ('test121', '6.047e-01')]

sorted_qfim_mineig:
 - min: 
  test32: rank = 2, min eigenvalue = 3.8748316910641734e-07
  test143: rank = 3, min eigenvalue = 8.285804824481602e-07
  test120: rank = 3, min eigenvalue = 2.601603910079575e-06
 - max: 
  test100: rank = 3, min eigenvalue = 0.04975012

In [75]:
number_of_tests

200

In [76]:
print("sorted_qfim_mineig:\n - min: ")
for key, (rnk, mineig) in sorted_qfim_mineig[:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")
print(f" - max: ")
for key, (rnk, mineig) in sorted_qfim_mineig[::-1][:num_print]:
    print(f"  {key}: rank = {rnk}, min eigenvalue = {mineig}")

sorted_qfim_mineig:
 - min: 
  test32: rank = 2, min eigenvalue = 1.784063169907313e-05
  test143: rank = 3, min eigenvalue = 3.345400756415984e-08
  test120: rank = 3, min eigenvalue = 7.25559004877141e-07
 - max: 
  test170: rank = 3, min eigenvalue = 1.8117555379867554
  test48: rank = 3, min eigenvalue = 1.3264001607894897
  test100: rank = 3, min eigenvalue = 1.075181484222412


In [ ]:
['test170','test49','test48','test100']

In [85]:


# Existing dictionaries
params_sorted_by_qfim_trace = {}
params_sorted_by_qfim_eigvals = {}
params_sorted_by_qfim_var, params_sorted_by_qfim_nonzero_var = {}, {}
params_sorted_by_qfim_mineig = {}
params_sorted_by_qfim_entropy = {}
key_and_params = {}

# NEW dictionaries for IPR/Abbas or single-draw stats
params_sorted_by_ipr_raw = {}
params_sorted_by_ipr_norm = {}
params_sorted_by_abbas_raw = {}
params_sorted_by_abbas_norm = {}
params_sorted_rank = {}
params_sorted_spread_var, params_sorted_spread_mad = {}, {}

# NEW dictionaries for the additional normalized metrics:
params_sorted_by_var_all_normalized_by_param_count,params_sorted_by_var_all_normalized_by_rank = {},{}
params_sorted_by_var_nonzero_normalized_by_rank = {}
params_sorted_by_var_nonzero_log = {}
params_sorted_by_trace_normalized_by_param_count = {}

params_sorted_rank, params_sorted_deff= {},{}

for trainable_params_dict in test_data.keys():
    results = test_data[trainable_params_dict]
    qfi_eigvals = results['qfim_eigvals']
    stats = compute_single_draw_stats(qfi_eigvals,results['qfim'],  threshold=threshold, gamma=1.0, n=1)
    
    # Extract metrics from stats
    rank = stats["draw_rank"]
    deff = stats['d_eff']
    # print(deff)
    trace = stats["trace_eigenvalues"]
    variance = stats["var_all_eigenvalues"]
    variance_nonzero = stats["var_nonzero_eigenvalues"]
    ipr_raw = stats["ipr_deff_raw"]
    ipr_norm = stats["ipr_deff_norm"]
    abbas_raw = stats["abbas_deff_raw"]
    abbas_norm = stats["abbas_deff_norm"]
    spread_mad = stats["spread_metric_mad"]
    spread_var = stats["spread_metric_variance"]
    
    #  normalized metrics 
    var_all_norm = stats["var_all_normalized_by_param_count"]
    var_all_norm_rank = stats["var_all_normalized_by_rank"]
    var_nonzero_norm = stats["var_nonzero_normalized_by_rank"]
    var_nonzero_log = stats["var_nonzero_log"]
    trace_norm_param = stats["trace_normalized_by_param_count"]
    
    # Store existing metrics
    params_sorted_deff[trainable_params_dict] = deff
    params_sorted_by_qfim_trace[trainable_params_dict] = float(trace)
    params_sorted_by_qfim_var[trainable_params_dict] = float(variance)
    params_sorted_by_qfim_nonzero_var[trainable_params_dict] = float(variance_nonzero)
    params_sorted_by_qfim_mineig[trainable_params_dict] = float(stats["min_nonzero_eigenvalue"])
    params_sorted_spread_mad[trainable_params_dict] = spread_mad
    params_sorted_spread_var[trainable_params_dict] = spread_var
    
    params_sorted_by_ipr_raw[trainable_params_dict] = ipr_raw
    params_sorted_by_ipr_norm[trainable_params_dict] = ipr_norm
    params_sorted_by_abbas_raw[trainable_params_dict] = abbas_raw
    params_sorted_by_abbas_norm[trainable_params_dict] = abbas_norm
    params_sorted_rank[trainable_params_dict] = rank
    
    try:
        ent = float(np.mean(results['entropies']))
    except KeyError:
        ent = float(results['entropy'])
    params_sorted_by_qfim_entropy[trainable_params_dict] = ent
    
    key_and_params[trainable_params_dict] = results['trainable_params']
    
    # Store the new normalized metrics
    params_sorted_by_var_all_normalized_by_param_count[trainable_params_dict] = float(var_all_norm)
    params_sorted_by_var_all_normalized_by_rank[trainable_params_dict] = float(var_all_norm_rank)
    params_sorted_by_var_nonzero_normalized_by_rank[trainable_params_dict] = float(var_nonzero_norm)
    params_sorted_by_var_nonzero_log[trainable_params_dict] = float(var_nonzero_log)
    params_sorted_by_trace_normalized_by_param_count[trainable_params_dict] = float(trace_norm_param)

# print(params_sorted_by_qfim_entropy)

# print(key_and_params.keys())
number_of_tests = len(test_data)

sorted_qfim_entropy = sorted(params_sorted_by_qfim_entropy.items(), key=lambda x: round(float(x[1]), 6))
sorted_qfim_var = sorted(params_sorted_by_qfim_var.items(), key=lambda x: float(x[1]))
sorted_qfim_var_normalized_by_rank = sorted(params_sorted_by_var_all_normalized_by_rank.items(), key=lambda x: float(x[1]))
sorted_qfim_var_nonzero = sorted(params_sorted_by_qfim_nonzero_var.items(), key=lambda x: float(x[1]))
sorted_qfim_trace = sorted(params_sorted_by_qfim_trace.items(), key=lambda x: float(x[1]))
# params_sorted_by_abbas_raw
sorted_qfim_deff = sorted(params_sorted_deff.items(), key=lambda x: float(x[1]))
sorted_qfim_abbas_d_eff = sorted(params_sorted_by_abbas_raw.items(), key=lambda x: float(x[1]))
sorted_qfim_ipr_d_eff = sorted(params_sorted_by_ipr_raw.items(), key=lambda x: float(x[1]))
# Calculate the number of parameters to select
number_of_tests = len(test_data)
sorted_qfim_spread_var = sorted(params_sorted_spread_var.items(), key=lambda x: float(x[1]))
sorted_qfim_mineig = sorted(params_sorted_by_qfim_mineig.items(), key=lambda x: float(x[1]))
def round_tuples2(lst, ndigits=2):
    """Convert each (key, value) in lst to (key, formatted_value).
    
    If the rounded value is less than or equal to ndigits, it is formatted in scientific notation (e).
    Otherwise, it is formatted in fixed-point notation (f).
    """
    formatted = []
    for k, v in lst:
        val = round(float(v), ndigits)
        if val <= ndigits:
            formatted_value = f"{v:.{ndigits+1}e}"
        else:
            formatted_value = f"{val:.{ndigits}f}"
        formatted.append((k, formatted_value))
    return formatted

num_print = 3
print(f"sorted_qfim_trace: \n - min: {round_tuples2(sorted_qfim_trace)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_trace[::-1])[:num_print]}\n")

print(f"sorted_qfim_var: \n - min: {round_tuples2(sorted_qfim_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_var_normalized_by_rank[::-1])[:num_print]}\n")

print(f"sorted_qfim_mineig: \n - min: {round_tuples2(sorted_qfim_mineig)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_mineig[::-1])[:num_print]}\n")

print(f"sorted_qfim_spread_var: \n - min: {round_tuples2(sorted_qfim_spread_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_spread_var[::-1])[:num_print]}\n")

print(f"sorted_qfim_deff: \n - min: {round_tuples2(sorted_qfim_deff)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_deff[::-1])[:num_print]}\n")


sorted_qfim_trace: 
 - min: [('test32', '1.353e-01'), ('test120', '4.669e-01'), ('test75', '5.984e-01')]
 - max: [('test48', '23.44'), ('test49', '23.39'), ('test170', '20.74')]

sorted_qfim_var: 
 - min: [('test32', '2.615e-03'), ('test120', '2.847e-02'), ('test58', '4.871e-02')]
 - max: [('test49', '19.23'), ('test48', '16.74'), ('test50', '12.14')]

sorted_qfim_mineig: 
 - min: [('test143', '3.345e-08'), ('test120', '7.256e-07'), ('test3', '1.414e-06')]
 - max: [('test170', '1.812e+00'), ('test48', '1.326e+00'), ('test100', '1.075e+00')]

sorted_qfim_spread_var: 
 - min: [('test100', '9.063e-01'), ('test17', '1.078e+00'), ('test79', '1.228e+00')]
 - max: [('test143', '75.75'), ('test13', '51.38'), ('test120', '48.41')]

sorted_qfim_deff: 
 - min: [('test143', '5.001e-01'), ('test32', '5.001e-01'), ('test93', '5.007e-01')]
 - max: [('test17', '6.977e-01'), ('test79', '6.912e-01'), ('test63', '6.817e-01')]



In [77]:
print(f"sorted_qfim_trace: \n - min: {round_tuples2(sorted_qfim_trace)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_trace[::-1])[:num_print]}\n")

print(f"sorted_qfim_var: \n - min: {round_tuples2(sorted_qfim_var_normalized_by_rank)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_var_normalized_by_rank[::-1])[:num_print]}\n")

print(f"sorted_qfim_ipr_d_eff: \n - min: {round_tuples2(sorted_qfim_ipr_d_eff)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_ipr_d_eff[::-1])[:num_print]}\n")

print(f"sorted_qfim_spread_var: \n - min: {round_tuples2(sorted_qfim_spread_var)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_spread_var[::-1])[:num_print]}\n")

print(f"sorted_qfim_deff: \n - min: {round_tuples2(sorted_qfim_deff)[:num_print]}")
print(f" - max: {round_tuples2(sorted_qfim_deff[::-1])[:num_print]}\n")


sorted_qfim_trace: 
 - min: [('test32', '1.353e-01'), ('test120', '4.669e-01'), ('test75', '5.984e-01')]
 - max: [('test48', '23.44'), ('test49', '23.39'), ('test170', '20.74')]

sorted_qfim_var: 
 - min: [('test32', '1.308e-03'), ('test120', '9.491e-03'), ('test58', '1.624e-02')]
 - max: [('test49', '19.23'), ('test48', '16.74'), ('test50', '12.14')]

sorted_qfim_ipr_d_eff: 
 - min: [('test143', '1.000e+00'), ('test32', '1.000e+00'), ('test93', '1.002e+00')]
 - max: [('test17', '2.23'), ('test79', '2.15'), ('test58', '2.06')]

sorted_qfim_spread_var: 
 - min: [('test100', '9.063e-01'), ('test17', '1.078e+00'), ('test79', '1.228e+00')]
 - max: [('test143', '75.75'), ('test13', '51.38'), ('test120', '48.41')]

sorted_qfim_deff: 
 - min: [('test143', '5.001e-01'), ('test32', '5.001e-01'), ('test93', '5.007e-01')]
 - max: [('test17', '6.977e-01'), ('test79', '6.912e-01'), ('test63', '6.817e-01')]



In [99]:
metrics_sorted = {
    "trace": sorted_qfim_trace,
    "var": sorted_qfim_var_normalized_by_rank,
    # "spread_var": sorted_qfim_spread_var,
   
    # "d_eff": sorted_qfim_deff,
    
    "ipr_d_eff":sorted_qfim_ipr_d_eff,
    # "entropy":sorted_qfim_entropy
}
num_print = 1  # number of unique keys to select from each end for each metric
# ['test170','test49','test48','test100']
# Create an empty set to collect keys
completed =  ['test32','test143','test120','test58','test3','test93','test170','test49','test48','test100','test170', 'test193', 'test32', 'test120', 'test58', 'test48', 'test15', 'test143']
print(len(completed))
# completed = ['test170','test49','test48','test100']
# , 'test170', 'test193', 'test32', 'test120', 'test58', 'test48', 'test15', 'test143'
selected_from_all = set(completed)
print(len(selected_from_all))

for metric_name, sorted_list in metrics_sorted.items():
    if not sorted_list:
        print(f"No data for metric {metric_name}")
        continue  # Skip if the list is empty

    # --- Select lower end keys ---
    # lower_count = 0
    # i = 0
    # while lower_count < num_print and i < len(sorted_list):
    #     key_candidate = sorted_list[i][0]
    #     print(f"{metric_name}: {key_candidate}")
    #     if key_candidate not in selected_from_all:
    #         selected_from_all.add(key_candidate)
    #         lower_count += 1
    #     i += 1

    # --- Select upper end keys ---
    upper_count = 0
    j = len(sorted_list) - 1
    while upper_count < num_print and j >= 0:
        key_candidate = sorted_list[j][0]
        print(f"{metric_name}: {key_candidate}")
        if key_candidate not in selected_from_all:
            selected_from_all.add(key_candidate)
            upper_count += 1
        j -= 1

 

print("Selected keys from min, max, and middle of each metric:", selected_from_all)
print("Total number of selected keys:", len(selected_from_all))

18
12
trace: test9
var: test9
var: test48
var: test15
var: test79
ipr_d_eff: test193
ipr_d_eff: test1
Selected keys from min, max, and middle of each metric: {'test3', 'test49', 'test9', 'test1', 'test93', 'test100', 'test170', 'test193', 'test32', 'test120', 'test79', 'test58', 'test48', 'test15', 'test143'}
Total number of selected keys: 15


In [84]:
a = ['test32','test143','test120','test58','test3','test93']

for test in a:
    print(f"\n{test} [rank: {params_sorted_rank[test]}]:")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]:.3f}, min(lam): {params_sorted_by_qfim_mineig[test][1]:.3e}")
    print(f"Var: {params_sorted_by_var_all_normalized_by_rank[test]:.3f}, Var (nonzero included): {params_sorted_by_var_nonzero_normalized_by_rank[test]:.3f}")
    print(f"d_eff: {params_sorted_by_ipr_raw[test]:.3f}, log(var): {params_sorted_spread_var[test]:.3f}, ")


test32 [rank: 2]:
Trace: 0.135, min(lam): 1.784e-05
Var: 0.001, Var (nonzero included): 0.005
d_eff: 1.000, log(var): 39.906, 

test143 [rank: 3]:
Trace: 1.190, min(lam): 3.345e-08
Var: 0.067, Var (nonzero included): 0.157
d_eff: 1.000, log(var): 75.755, 

test120 [rank: 3]:
Trace: 0.467, min(lam): 7.256e-07
Var: 0.009, Var (nonzero included): 0.022
d_eff: 1.079, log(var): 48.411, 

test58 [rank: 3]:
Trace: 0.923, min(lam): 1.509e-02
Var: 0.016, Var (nonzero included): 0.022
d_eff: 2.058, log(var): 3.862, 

test3 [rank: 3]:
Trace: 0.935, min(lam): 1.414e-06
Var: 0.040, Var (nonzero included): 0.093
d_eff: 1.027, log(var): 46.689, 

test93 [rank: 3]:
Trace: 0.991, min(lam): 1.734e-06
Var: 0.047, Var (nonzero included): 0.109
d_eff: 1.002, log(var): 43.974, 


In [81]:
b = ['test170','test49','test48','test100']

for test in b:
    print(f"\n{test} [rank: {params_sorted_rank[test]}]:")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]:.3f}, min(lam): {params_sorted_by_qfim_mineig[test][1]:.3e}")
    print(f"Var: {params_sorted_by_var_all_normalized_by_rank[test]:.3f}, Var (nonzero included): {params_sorted_by_var_nonzero_normalized_by_rank[test]:.3f}")
    print(f"d_eff: {params_sorted_by_ipr_raw[test]:.3f}, log(var): {params_sorted_spread_var[test]:.3f}, ")


test170 [rank: 3]:
Trace: 20.742, min(lam): 1.812e+00
Var: 11.651, Var (nonzero included): 21.295
d_eff: 1.586, log(var): 1.338, 

test49 [rank: 3]:
Trace: 23.395, min(lam): 9.898e-01
Var: 19.234, Var (nonzero included): 40.326
d_eff: 1.290, log(var): 2.538, 

test48 [rank: 3]:
Trace: 23.439, min(lam): 1.326e+00
Var: 16.735, Var (nonzero included): 32.765
d_eff: 1.447, log(var): 1.891, 

test100 [rank: 3]:
Trace: 10.505, min(lam): 1.075e+00
Var: 2.307, Var (nonzero included): 3.417
d_eff: 1.926, log(var): 0.906, 


In [69]:
b = ['test49','test48','test16']

for test in b:
    print(f"\n{test} [rank: {params_sorted_rank[test]}]:")
    
    print(f"Trace: {params_sorted_by_qfim_trace[test]:.3f}, min(lam): {params_sorted_by_qfim_mineig[test][1]:.3e}")
    print(f"Var: {params_sorted_by_var_all_normalized_by_rank[test]:.3f}, Var (nonzero included): {params_sorted_by_var_nonzero_normalized_by_rank[test]:.3f}")
    print(f"d_eff: {params_sorted_by_ipr_raw[test]:.3f}, log(var): {params_sorted_spread_var[test]:.3f}, ")


test49 [rank: 3]:
Trace: 23.395, min(lam): 9.898e-01
Var: 19.234, Var (nonzero included): 40.326
d_eff: 1.290, log(var): 2.538, 

test48 [rank: 3]:
Trace: 23.439, min(lam): 1.326e+00
Var: 16.735, Var (nonzero included): 32.765
d_eff: 1.447, log(var): 1.891, 

test16 [rank: 3]:
Trace: 9.629, min(lam): 5.346e-01
Var: 2.094, Var (nonzero included): 3.339
d_eff: 1.820, log(var): 1.617, 


In [115]:
import math
import numpy as np
import pandas as pd

# ----------------------------------------------------------------------
# 1. Compute single-draw QFIM/DQFIM statistics (unchanged)
# ----------------------------------------------------------------------
def compute_single_draw_stats(
    eigvals,
    full_qfim_mat,
    threshold=1e-10,
    spread_methods=("variance", "mad"),
    ddof=1,
    scale="normal",
    gamma=1.0,
    n=1,
    V_theta=1.0,
    n_ctrl=None,
    n_reserv=None,
    trotter_step=None,
):
    """
    Compute QFIM (or DQFIM) statistics for a SINGLE set of eigenvalues.
    (This function is unchanged.)
    """
    arr = np.array(eigvals)
    if arr.ndim != 1:
        arr = arr.flatten()
    arr = np.where(arr < threshold, 0.0, arr)
    
    draw_rank = np.count_nonzero(arr)
    var_all_eigenvalues = np.var(arr, ddof=ddof)
    nonzero = arr[arr > threshold]
    var_nonzero_eigenvalues = np.var(nonzero, ddof=ddof) if nonzero.size > 1 else 0.0
    var_nonzero_log = np.log(var_nonzero_eigenvalues) if var_nonzero_eigenvalues > 0 else -np.inf
    trace_eigenvalues = np.sum(arr)
    min_nonzero_eigenvalue = np.min(nonzero)
    
    var_normalized_by_param_count = var_all_eigenvalues / len(arr)
    var_nonzero_normalized_by_rank = var_nonzero_eigenvalues / draw_rank if draw_rank > 0 else 0.0
    var_normalized_by_rank = var_all_eigenvalues / draw_rank if draw_rank > 0 else 0.0
    trace_normalized_by_rank = (trace_eigenvalues / draw_rank) if draw_rank > 0 else 0.0
    trace_normalized_by_param_count = trace_eigenvalues / len(arr)
    
    sum_of_squares = np.sum(arr**2)
    ipr_deff_raw = (trace_eigenvalues**2) / sum_of_squares if sum_of_squares > 0 else 0.0
    
    if trace_eigenvalues > 0:
        arr_norm = arr / trace_eigenvalues
        sum_norm_sq = np.sum(arr_norm**2)
        ipr_deff_norm = 1.0 / sum_norm_sq if sum_norm_sq > 0 else 0.0
    else:
        arr_norm = None
        ipr_deff_norm = 0.0

    if n > 1 and math.log(n) != 0.0:
        alpha = (gamma * n) / (2.0 * math.log(n))
    else:
        alpha = 0.0
    abbas_deff_raw = np.sum(np.log(np.maximum(1.0 + alpha * arr, 1e-15)))
    if arr_norm is not None:
        abbas_deff_norm = np.sum(np.log(np.maximum(1.0 + alpha * arr_norm, 1e-15)))
    else:
        abbas_deff_norm = 0.0

    F = np.array(full_qfim_mat, dtype=complex)
    trF = np.trace(F)
    if trF > 0:
        F_hat = F / trF
        eigs_F = np.linalg.eigvalsh(F_hat)
        eps = 1e-12
        if n > 1 and math.log(n) != 0.0:
            z = 0.5 * np.sum(np.log(1.0 + n * eigs_F + eps))
            effective_dimension = (2.0 / np.log(n)) * z
        else:
            effective_dimension = np.sum(eigs_F / (1.0 + eigs_F))
    else:
        effective_dimension = 0.0

    arr_2d = arr.reshape(1, -1)
    spread_metrics = {}
    # (Assuming spread_per_sample_vectorized is defined elsewhere.)
    for method in spread_methods:
        per_draw = spread_per_sample_vectorized(arr_2d, method=method, threshold=threshold, ddof=ddof, scale=scale)
        spread_metrics[f"spread_metric_{method}"] = per_draw[0] if per_draw.size > 0 else 0.0
    
    stats_dict = {
        "draw_rank": draw_rank,
        "var_all_eigenvalues": var_all_eigenvalues,
        "var_nonzero_eigenvalues": var_nonzero_eigenvalues,
        "trace_eigenvalues": trace_eigenvalues,
        "var_all_normalized_by_param_count": var_normalized_by_param_count,
        "var_all_normalized_by_rank": var_normalized_by_rank,
        "var_nonzero_normalized_by_rank": var_nonzero_normalized_by_rank,
        "trace_normalized_by_rank": trace_normalized_by_rank,
        "trace_normalized_by_param_count": trace_normalized_by_param_count,
        "var_nonzero_log": var_nonzero_log,
        "ipr_deff_raw": ipr_deff_raw,
        "ipr_deff_norm": ipr_deff_norm,
        "abbas_deff_raw": abbas_deff_raw,
        "abbas_deff_norm": abbas_deff_norm,
        "d_eff": effective_dimension,
        # Minimum nonzero eigenvalue (above threshold)
        "min_nonzero_eigenvalue": min_nonzero_eigenvalue,
    }
    stats_dict.update(spread_metrics)
    
    return stats_dict

# ----------------------------------------------------------------------
# 2. clean_array (unchanged)
# ----------------------------------------------------------------------
def clean_array(data):
    if isinstance(data, np.ndarray):
        return np.array(data)
    elif isinstance(data, dict):
        return {k: clean_array(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [clean_array(v) for v in data]
    else:
        return data

# ----------------------------------------------------------------------
# 3. Updated read_jax_file
# ----------------------------------------------------------------------
def read_jax_file(file_path, gate_name, test_key,num_L):
    """
    Read the pickle file and extract fields using the updated keys.
    Now extracts:
      - QFIM Results (GHZ)
      - QFIM_basis_state
      - Computed training DQFIM from "DQFIM_stats_local"
      - Original (file-stored) DQFIM from "DQFIM_stats_{NUM_L}_L_states"
      - Computed target DQFIM from "target DQFIM stats"
    """
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
    df = clean_array(df)
    
    try:
        costs = np.asarray([float(i) for i in df['costs'][0]], dtype=np.float64)
        N_train = float(df['N_train'][0])
    except Exception as e:
        print(f"Error reading costs/N_train from {file_path}: {e}")
        costs = None
        N_train= None
        
    try:
        grads_per_epoch = [np.asarray(i, dtype=np.float64) for i in df['grads_per_epoch'][0]]
    except Exception as e:
        print(f"Error reading grads_per_epoch from {file_path}: {e}")
        grads_per_epoch = None
        
    try:
        fidelity = float(df['avg_fidelity'][0])
    except Exception as e:
        print(f"Error reading avg_fidelity from {file_path}: {e}")
        fidelity = None
        
    try:
        num_params = 3 + int(df['controls'][0]) * int(df['reservoirs'][0]) * int(df['trotter_step'][0]) + int(df['trotter_step'][0])
    except Exception as e:
        print(f"Error computing num_params from {file_path}: {e}")
        num_params = None
        
    try:
        test_results = np.asarray(df['testing_results'][0], dtype=np.float64)
    except Exception as e:
        print(f"Error reading testing_results from {file_path}: {e}")
        test_results = None
        
    # QFIM Results (GHZ)
    qfim_stats_dict_GHZ = df.get('QFIM Results', [None])[0]
    if qfim_stats_dict_GHZ is None:
        print(f"Warning: 'QFIM Results' not found in {file_path}")
        qfim_eigvals_GHZ = qfim_full_GHZ = entropy_GHZ = None
    else:
        qfim_eigvals_GHZ = qfim_stats_dict_GHZ.get('qfim_eigvals', None)
        qfim_full_GHZ = qfim_stats_dict_GHZ.get('qfim', None)
        entropy_GHZ = qfim_stats_dict_GHZ.get('entropy', None)
    
    # QFIM Results (GHZ)
    qfim_stats_dict_GHZ = df['QFIM Results'][0]
    qfim_eigvals_GHZ = qfim_stats_dict_GHZ['qfim_eigvals']
    qfim_full_GHZ = qfim_stats_dict_GHZ['qfim']
    entropy_GHZ = qfim_stats_dict_GHZ['entropy']
    
    # QFIM Basis State
    qfim_stats_dict_basis = df['QFIM_basis_state'][0]
    qfim_eigvals_basis = qfim_stats_dict_basis['qfim_eigvals']
    qfim_full_basis = qfim_stats_dict_basis['qfim']
    entropy_basis = qfim_stats_dict_basis['entropy']
    
    # Computed Training DQFIM stats (new key)
    dqfim_stats_local = df['DQFIM_stats_local'][0]
    dqfim_eigvals_train = dqfim_stats_local['eigvals_train']
    dqfim_train = dqfim_stats_local['DQFIM_train']
    dqfim_entropies = dqfim_stats_local['entropies_train']
    
    # Original (file-stored) DQFIM stats (new key) – stored under key like "DQFIM_stats_{NUM_L}_L_states"
    key_name = f'DQFIM_stats_{num_L}_L_states'
    dqfim_file_stats = df[key_name][0]
    dqfim_eigvals_file = dqfim_file_stats['dqfim_eigvals']
    dqfim_full_file = dqfim_file_stats['dqfim']
   
    # Computed Target DQFIM stats (new key) target_dqfim_entropies
    computed_target = df["target DQFIM stats"][0]
    target_dqfim_eigvals = computed_target['eigvals_target']
    target_dqfim_full = computed_target['DQFIM_target']
    target_dqfim_entropies = computed_target['entropies_target']
                                             
    readin_test_key = df.get("test_key", [None])[0]
    assert readin_test_key == test_key, f'Got: {readin_test_key}. Expected: {test_key}'
    result = {
        "costs": costs,
        "fidelity": fidelity,
        "num_params": num_params,
        "test_results": test_results,
        "qfim_eigvals_GHZ": qfim_eigvals_GHZ,
        "qfim_full_GHZ": qfim_full_GHZ,
        "entropy_GHZ": entropy_GHZ,
        "qfim_eigvals_basis": qfim_eigvals_basis,
        "qfim_full_basis": qfim_full_basis,
        "entropy_basis": entropy_basis,
        "dqfim_eigvals_train": dqfim_eigvals_train,
        "dqfim_train": dqfim_train,
        "dqfim_entropies": dqfim_entropies,
        "dqfim_eigvals_file": dqfim_eigvals_file,
        "dqfim_full_file": dqfim_full_file,
        "num_sampled_states": num_L,
        "target_dqfim_eigvals": target_dqfim_eigvals,
        "target_dqfim_full": target_dqfim_full,
        "target_dqfim_entropies": target_dqfim_entropies,
        "N_ctrl": df.get('controls', [None])[0],
        "Trotter_Step": df.get('trotter_step', [None])[0],
        "N_R": df.get('reservoirs', [None])[0],
        "gate": gate_name,
        'N_train':N_train,
        "test_key": readin_test_key
    }
    return result

# ----------------------------------------------------------------------
# 4. Build and aggregate DataFrame results (updated)
# ----------------------------------------------------------------------
def build_df_results(fixed_param_folder, base_folder, N_C, N_R, T,num_L):
    """
    Build a DataFrame by scanning the results folder.
    Updated to include new keys (e.g., 'DQFIM_stats_local' and original file DQFIM stats).
    """
    rows = []
    full_path = os.path.join(base_folder, fixed_param_folder)
    for test_key in os.listdir(full_path):
        test_key_path = os.path.join(full_path, test_key)
        if not os.path.isdir(test_key_path):
            continue
        for gate_folder in os.listdir(test_key_path):
            if not gate_folder.startswith(f"U{N_C}_"):
                continue
            gate_folder_path = os.path.join(test_key_path, gate_folder)
            if not os.path.isdir(gate_folder_path):
                continue
            datarun_files_in_folder = os.listdir(gate_folder_path)
            if len(datarun_files_in_folder) > 1:
                file_name = f"data_run_{len(datarun_files_in_folder)-1}.pickle"
                print(f'{test_key} gate {gate_folder} has {len(datarun_files_in_folder)} data runs stored. Picking latest one: {file_name}')
            else:
                file_name = "data_run_0.pickle"
            pickle_file = os.path.join(gate_folder_path, file_name)
            if os.path.isfile(pickle_file):
                try:
                    data = read_jax_file(pickle_file, gate_folder, test_key, num_L)
                    data["test_key"] = test_key
                    data["gate_folder"] = gate_folder
                    data["file_path"] = pickle_file
                    rows.append(data)
                except Exception as ex:
                    print(f"Error processing {pickle_file}: {ex}")
                    raise  # Break out immediately on error.
            else:
                print(f"Pickle file does not exist: {pickle_file}")
    df_results = pd.DataFrame(rows)
    return df_results

# Example usage in post-processing:
# df_results = build_df_results(fixed_param_folder, base_folder, N_C=2)
# df_agg = aggregate_results(df_results)
# df_final = update_with_all_qfim_metrics(df_agg)
# print(df_final.shape)

trotter_Step = 2
N_ctrl = 1
num_L = 10
N_reserv = 1
fixed_param_folder = "fixed_params0"
num_epochs = 1000
train_size = 10
sample_range_label = 'normal_.25pi'
base_folder = f"/Users/sophieblock/QRCCapstone/parameter_analysis_directory/param_initialization_final/analog_results/Nc_{N_ctrl}/epochs_{num_epochs}/reservoirs_{N_reserv}/trotter_{trotter_Step}/trainsize_{train_size}/sample_{sample_range_label}"
df_results = build_df_results(fixed_param_folder, base_folder, N_C=N_ctrl, N_R=N_reserv, T=trotter_Step,num_L=num_L)

print("df_results shape:", df_results.shape)


test32 gate U1_2 has 2 data runs stored. Picking latest one: data_run_1.pickle
test32 gate U1_0 has 3 data runs stored. Picking latest one: data_run_2.pickle
test32 gate U1_1 has 3 data runs stored. Picking latest one: data_run_2.pickle
test143 gate U1_3 has 2 data runs stored. Picking latest one: data_run_1.pickle
test143 gate U1_4 has 2 data runs stored. Picking latest one: data_run_1.pickle
test143 gate U1_2 has 2 data runs stored. Picking latest one: data_run_1.pickle
test143 gate U1_0 has 3 data runs stored. Picking latest one: data_run_2.pickle
test143 gate U1_1 has 3 data runs stored. Picking latest one: data_run_2.pickle
test93 gate U1_2 has 2 data runs stored. Picking latest one: data_run_1.pickle
test93 gate U1_0 has 3 data runs stored. Picking latest one: data_run_2.pickle
test93 gate U1_1 has 2 data runs stored. Picking latest one: data_run_1.pickle
test58 gate U1_3 has 2 data runs stored. Picking latest one: data_run_1.pickle
test58 gate U1_2 has 2 data runs stored. Pickin

In [131]:

def aggregate_results(df):
    """
    Aggregate the DataFrame by test_key and gate.
    Now also includes the original DQFIM file stats.
    """
    aggregated = df.groupby(["test_key", "gate"]).agg(
        fidelities_list=("fidelity", list),
        avg_fidelity=("fidelity", "mean"),
        error=("fidelity", lambda x: np.mean(np.log(1 - x))),
        avg_infidelity=("fidelity", lambda x: np.mean(1 - x)),
        qfim_eigvals_GHZ=("qfim_eigvals_GHZ", "first"),
        qfim_full_GHZ=("qfim_full_GHZ", "first"),
        entropy_GHZ=("entropy_GHZ", "first"),
        qfim_eigvals_basis=("qfim_eigvals_basis", "first"),
        qfim_full_basis=("qfim_full_basis", "first"),
        entropy_basis=("entropy_basis", "first"),
        dqfim_eigvals_train=("dqfim_eigvals_train", "first"),
        dqfim_train=("dqfim_train", "first"),
        dqfim_eigvals_file=("dqfim_eigvals_file", "first"),
        dqfim_full_file=("dqfim_full_file", "first"),
        target_dqfim_eigvals=("target_dqfim_eigvals", "first"),
        target_dqfim_full=("target_dqfim_full", "first"),
        target_dqfim_entropies=("target_dqfim_entropies", "first"),
        N_ctrl=("N_ctrl", "first"),
        N_R=("N_R", "first"),
        Trotter_Step=("Trotter_Step", "first"),
        num_sampled_states=("num_sampled_states", "first"),
        num_train = ("N_train","first")
    ).reset_index()
    return aggregated

def update_with_all_qfim_metrics(df, threshold=1e-10, spread_methods=("variance", "mad"),
                                 ddof=1, scale="normal", gamma=0.1, n=1, V_theta=1.0):
    """
    For each row, compute derived metrics for each QFIM variant:
      - GHZ QFIM (from "qfim_eigvals_GHZ" and "qfim_full_GHZ")
      - Basis QFIM (from "qfim_eigvals" and "qfim_full")
      - Computed Training DQFIM (from "dqfim_eigvals_train" and "dqfim_train")
      - Original file-stored DQFIM (from "dqfim_eigvals_file" and "dqfim_full_file")
      - Computed Target DQFIM (from "target_dqfim_eigvals" and "target_dqfim_full")
    """
    new_rows = []
    for _, row in df.iterrows():
        ghz_stats = compute_single_draw_stats(
            row["qfim_eigvals_GHZ"],
            row["qfim_full_GHZ"],
            threshold=threshold,
            spread_methods=spread_methods,
            ddof=ddof,
            scale=scale,
            gamma=gamma,
            n=1,
            V_theta=V_theta,
            n_ctrl=row["N_ctrl"],
            n_reserv=row["N_R"],
            trotter_step=row["Trotter_Step"]
        )
        basis_stats = compute_single_draw_stats(
            row["qfim_eigvals_basis"],
            row["qfim_full_basis"],
            threshold=threshold,
            spread_methods=spread_methods,
            ddof=ddof,
            scale=scale,
            gamma=gamma,
            n=1,
            V_theta=V_theta,
            n_ctrl=row["N_ctrl"],
            n_reserv=row["N_R"],
            trotter_step=row["Trotter_Step"]
        )
        dqfim_stats_train = compute_single_draw_stats(
            row["dqfim_eigvals_train"],
            row["dqfim_train"],
            threshold=threshold,
            spread_methods=spread_methods,
            ddof=ddof,
            scale=scale,
            gamma=gamma,
            n=row['num_train'] if 'num_train' in row else 1,
            V_theta=V_theta,
            n_ctrl=row["N_ctrl"],
            n_reserv=row["N_R"],
            trotter_step=row["Trotter_Step"]
        )
        file_dqfim_stats = compute_single_draw_stats(
            row["dqfim_eigvals_file"],
            row["dqfim_full_file"],
            threshold=threshold,
            spread_methods=spread_methods,
            ddof=ddof,
            scale=scale,
            gamma=gamma,
            n=row['num_sampled_states'] if 'num_sampled_states' in row else 1,
            V_theta=V_theta,
            n_ctrl=row["N_ctrl"],
            n_reserv=row["N_R"],
            trotter_step=row["Trotter_Step"]
        )
        dqfim_stats_targ = compute_single_draw_stats(
            row["target_dqfim_eigvals"],
            row["target_dqfim_full"],
            threshold=threshold,
            spread_methods=spread_methods,
            ddof=ddof,
            scale=scale,
            gamma=gamma,
            n=row['num_train'] if 'num_train' in row else 1,
            V_theta=V_theta,
            n_ctrl=row["N_ctrl"],
            n_reserv=row["N_R"],
            trotter_step=row["Trotter_Step"]
        )
        updated_row = row.to_dict()
        updated_row.update({f"GHZ_{k}": v for k, v in ghz_stats.items()})
        updated_row.update({f"basis_{k}": v for k, v in basis_stats.items()})
        updated_row.update({f"dqfim_{k}": v for k, v in dqfim_stats_train.items()})
        updated_row.update({f"random_dample_dqfim_{k}": v for k, v in file_dqfim_stats.items()})
        updated_row.update({f"tdqfim_{k}": v for k, v in dqfim_stats_targ.items()})
        new_rows.append(updated_row)
    return pd.DataFrame(new_rows)


In [132]:
df_agg = aggregate_results(df_results)
print("Aggregated DataFrame shape:", df_agg.shape)
display(df_agg.keys())


df_final = update_with_all_qfim_metrics(df_agg)
df_final.shape
display(df_final.keys())


Aggregated DataFrame shape: (240, 24)


Index(['test_key', 'gate', 'fidelities_list', 'avg_fidelity', 'error',
       'avg_infidelity', 'qfim_eigvals_GHZ', 'qfim_full_GHZ', 'entropy_GHZ',
       'qfim_eigvals_basis', 'qfim_full_basis', 'entropy_basis',
       'dqfim_eigvals_train', 'dqfim_train', 'dqfim_eigvals_file',
       'dqfim_full_file', 'target_dqfim_eigvals', 'target_dqfim_full',
       'target_dqfim_entropies', 'N_ctrl', 'N_R', 'Trotter_Step',
       'num_sampled_states', 'num_train'],
      dtype='object')

Index(['test_key', 'gate', 'fidelities_list', 'avg_fidelity', 'error',
       'avg_infidelity', 'qfim_eigvals_GHZ', 'qfim_full_GHZ', 'entropy_GHZ',
       'qfim_eigvals_basis',
       ...
       'tdqfim_trace_normalized_by_param_count', 'tdqfim_var_nonzero_log',
       'tdqfim_ipr_deff_raw', 'tdqfim_ipr_deff_norm', 'tdqfim_abbas_deff_raw',
       'tdqfim_abbas_deff_norm', 'tdqfim_d_eff',
       'tdqfim_min_nonzero_eigenvalue', 'tdqfim_spread_metric_variance',
       'tdqfim_spread_metric_mad'],
      dtype='object', length=114)

In [129]:
import pingouin as pg
import warnings
from scipy.stats import ConstantInputWarning
def analyze_correlations(df_merged, x_metric, metrics_of_interest, corr_threshold=0.2, p_threshold=0.05,
                         print_all_pearson=False, print_all_spearman=False):
    """
    Analyze pairwise correlations between a given x_metric and each metric in metrics_of_interest.
    
    Parameters
    ----------
    df_merged : pd.DataFrame
        The merged DataFrame containing the columns of interest.
    x_metric : str
        The column name for the independent variable (e.g., "avg_fidelity").
    metrics_of_interest : list of str
        List of column names whose correlations with x_metric will be computed.
    corr_threshold : float, optional
        Minimum absolute correlation coefficient to report (default 0.2).
    p_threshold : float, optional
        Maximum p-value threshold to report (default 0.05).
    print_all_pearson : bool, optional
        If True, print Pearson correlation results for every metric.
    print_all_spearman : bool, optional
        If True, print Spearman correlation results for every metric.
        
    Returns
    -------
    pearson_results : dict
        Dictionary mapping each metric to its Pearson correlation coefficient and p-value.
    spearman_results : dict
        Dictionary mapping each metric to its Spearman correlation coefficient and p-value.
    """
    # Create a new DataFrame with the columns of interest and drop rows with NaN values.
    df_corr = df_merged[[x_metric] + metrics_of_interest].copy()
    df_corr = df_corr.dropna(subset=[x_metric] + metrics_of_interest)
    
    # Ensure that each metric column contains float values.
    for col in metrics_of_interest:
        df_corr[col] = df_corr[col].apply(
            lambda val: float(val.item()) if hasattr(val, "item") else float(val)
        )
    
    # Compute Pearson correlations.
    pearson_results = {}
    for col in metrics_of_interest:
        try:
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("error", category=ConstantInputWarning)
                res_df = pg.corr(x=df_corr[x_metric], y=df_corr[col], method="pearson")
            r_val = res_df["r"].iloc[0]
            p_val = res_df["p-val"].iloc[0]
            pearson_results[col] = {"pearson_r": r_val, "p_value": p_val}
        except ConstantInputWarning as cie:
            print(f"ConstantInputWarning for {col} (Pearson): {cie}")
            pearson_results[col] = {"pearson_r": None, "p_value": None}
    
    # Compute Spearman correlations.
    spearman_results = {}
    for col in metrics_of_interest:
        try:
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("error", category=ConstantInputWarning)
                sp_df = pg.corr(x=df_corr[x_metric], y=df_corr[col], method="spearman")
            rho_val = sp_df["r"].iloc[0]
            p_val = sp_df["p-val"].iloc[0]
            spearman_results[col] = {"spearman_rho": rho_val, "p_value": p_val}
        except ConstantInputWarning as cie:
            print(f"ConstantInputWarning for {col} (Spearman): {cie}")
            spearman_results[col] = {"spearman_rho": None, "p_value": None}
    
    # Print out the Pearson correlations.
    print(f"\nPairwise correlations vs. {x_metric} (Pearson):")
    for metric, vals in pearson_results.items():
        if print_all_pearson or (vals["pearson_r"] is not None and abs(vals["pearson_r"]) > corr_threshold and vals["p_value"] < p_threshold):
            if vals["pearson_r"] is not None:
                print(f"{metric}: r = {vals['pearson_r']:.3f}, p = {vals['p_value']:.3g}")
            else:
                print(f"{metric}: r = None, p = None")
    
    # Print out the Spearman correlations.
    print(f"\nPairwise correlations vs. {x_metric} (Spearman):")
    for metric, vals in spearman_results.items():
        if print_all_spearman or (vals["spearman_rho"] is not None and abs(vals["spearman_rho"]) > corr_threshold and vals["p_value"] < p_threshold):
            if vals["spearman_rho"] is not None:
                print(f"{metric}: rho = {vals['spearman_rho']:.3f}, p = {vals['p_value']:.3g}")
            else:
                print(f"{metric}: rho = None, p = None")
    
    return pearson_results, spearman_results



CORR_THRESHOLD = 0.1
P_THRESHOLD = 0.05
x_metric = "avg_fidelity"
metrics_of_interest_ghz = [

  
       'GHZ_var_all_normalized_by_rank', 'GHZ_var_nonzero_normalized_by_rank',
       'GHZ_trace_normalized_by_rank', 
       'GHZ_var_nonzero_log',  'GHZ_ipr_deff_norm', 'GHZ_d_eff',
       'GHZ_spread_metric_variance', 'GHZ_spread_metric_mad',
]
# Now you can use these common thresholds in your calls:
pearson_corrs_ghz, spearman_corrs_ghz = analyze_correlations(
    df_final, x_metric, metrics_of_interest_ghz,
    corr_threshold=CORR_THRESHOLD, p_threshold=P_THRESHOLD
)

# for metric, vals in pearson_corrss
# on_r']:.3f}, p={vals['p_value']:.3g}")
    


Pairwise correlations vs. avg_fidelity (Pearson):
GHZ_var_all_normalized_by_rank: r = -0.158, p = 0.0142
GHZ_var_nonzero_normalized_by_rank: r = -0.154, p = 0.0171
GHZ_trace_normalized_by_rank: r = -0.145, p = 0.0252

Pairwise correlations vs. avg_fidelity (Spearman):
GHZ_trace_normalized_by_rank: rho = -0.140, p = 0.0304


In [139]:
tdqfim_metrics = [key for key in df_final.keys() if key.startswith('dqfim_')]
print("Target dQFIM metric keys:", tdqfim_metrics)

Target dQFIM metric keys: ['dqfim_eigvals_train', 'dqfim_train', 'dqfim_eigvals_file', 'dqfim_full_file', 'dqfim_draw_rank', 'dqfim_var_all_eigenvalues', 'dqfim_var_nonzero_eigenvalues', 'dqfim_trace_eigenvalues', 'dqfim_var_all_normalized_by_param_count', 'dqfim_var_all_normalized_by_rank', 'dqfim_var_nonzero_normalized_by_rank', 'dqfim_trace_normalized_by_rank', 'dqfim_trace_normalized_by_param_count', 'dqfim_var_nonzero_log', 'dqfim_ipr_deff_raw', 'dqfim_ipr_deff_norm', 'dqfim_abbas_deff_raw', 'dqfim_abbas_deff_norm', 'dqfim_d_eff', 'dqfim_min_nonzero_eigenvalue', 'dqfim_spread_metric_variance', 'dqfim_spread_metric_mad']


updated_row.update({f"GHZ_{k}": v for k, v in ghz_stats.items()})
updated_row.update({f"basis_{k}": v for k, v in basis_stats.items()})
updated_row.update({f"dqfim_{k}": v for k, v in dqfim_stats_train.items()})
updated_row.update({f"random_dample_dqfim_{k}": v for k, v in file_dqfim_stats.items()})
updated_row.update({f"tdqfim_{k}": v for k, v in dqfim_stats_targ.items()})

In [138]:
x_metric = 'avg_fidelity'
metrics_of_interest_targ_dqfim = [
    
       'tdqfim_var_all_eigenvalues',
       'tdqfim_var_nonzero_eigenvalues',
       'tdqfim_var_all_normalized_by_rank',
       'tdqfim_var_nonzero_log',
      'tdqfim_ipr_deff_raw', 'tdqfim_abbas_deff_raw', 
      'tdqfim_abbas_deff_norm', 'tdqfim_d_eff', 'tdqfim_min_nonzero_eigenvalue',
      'tdqfim_spread_metric_variance', 'tdqfim_spread_metric_mad']
pearson_corrs_targ, spearman_corrs_targ = analyze_correlations(
    df_final, x_metric, metrics_of_interest_targ_dqfim,
    corr_threshold=CORR_THRESHOLD, p_threshold=P_THRESHOLD, print_all_pearson=True
)


Pairwise correlations vs. avg_fidelity (Pearson):
tdqfim_var_all_eigenvalues: r = -0.125, p = 0.0536
tdqfim_var_nonzero_eigenvalues: r = -0.117, p = 0.0692
tdqfim_var_all_normalized_by_rank: r = -0.125, p = 0.0536
tdqfim_var_nonzero_log: r = -0.112, p = 0.0827
tdqfim_ipr_deff_raw: r = 0.068, p = 0.293
tdqfim_abbas_deff_raw: r = -0.111, p = 0.0855
tdqfim_abbas_deff_norm: r = 0.036, p = 0.579
tdqfim_d_eff: r = 0.011, p = 0.866
tdqfim_min_nonzero_eigenvalue: r = -0.114, p = 0.0771
tdqfim_spread_metric_variance: r = -0.003, p = 0.966
tdqfim_spread_metric_mad: r = 0.007, p = 0.911

Pairwise correlations vs. avg_fidelity (Spearman):


In [141]:

metrics_of_interest_dqfim = [

     
       'dqfim_var_all_normalized_by_rank',
       'dqfim_var_nonzero_normalized_by_rank',
       'dqfim_trace_normalized_by_rank',
       'dqfim_var_nonzero_log',
     'dqfim_ipr_deff_norm', 'dqfim_abbas_deff_raw',
       'dqfim_abbas_deff_norm', 'dqfim_d_eff', 'dqfim_spread_metric_variance',
       'dqfim_spread_metric_mad'
]
pearson_corrs_dqfim, spearman_corrs_dqfim = analyze_correlations(
    df_final, x_metric, metrics_of_interest_dqfim,
    corr_threshold=CORR_THRESHOLD, p_threshold=P_THRESHOLD, print_all_pearson=True
)


Pairwise correlations vs. avg_fidelity (Pearson):
dqfim_var_all_normalized_by_rank: r = -0.028, p = 0.671
dqfim_var_nonzero_normalized_by_rank: r = -0.023, p = 0.722
dqfim_trace_normalized_by_rank: r = -0.064, p = 0.327
dqfim_var_nonzero_log: r = -0.095, p = 0.144
dqfim_ipr_deff_norm: r = -0.052, p = 0.426
dqfim_abbas_deff_raw: r = -0.074, p = 0.256
dqfim_abbas_deff_norm: r = -0.062, p = 0.341
dqfim_d_eff: r = -0.061, p = 0.346
dqfim_spread_metric_variance: r = 0.062, p = 0.34
dqfim_spread_metric_mad: r = 0.057, p = 0.38

Pairwise correlations vs. avg_fidelity (Spearman):
